Imports:

In [886]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, Normalizer

In [887]:
def distance_into_probability(x, w, b):
    ''' Функция возвращает по найденному уравнению разделяющей плоскости вероятность объекта быть идеальным
    x - это признаки данного объекта
    b - вектор коэффициентов в уравнении плоскости '''
    lin_comb = b
    for i in range(len(x)):
        lin_comb += x[i] * w[i]
    return 1 / (1 + np.exp(-lin_comb))

Делим исходные данные на признаки и ответы, а также разделяем на данные, на которых обучаем и на которых тестируем

In [888]:
df = pd.read_csv('../assets/train.csv')

heads = df.keys()[:-1]

X = df[heads]
y = df['result']

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [889]:
scaler = StandardScaler().fit(x_train) 
standardized_X = scaler.transform(x_train)

x_train = standardized_X

In [890]:
scaler = StandardScaler().fit(x_test) 
standardized_X = scaler.transform(x_test)

x_test = standardized_X

In [891]:
model = svm.SVC(kernel='linear', C = 1.0, probability=True)

model.fit(x_train, y_train)
w_coeffs = list(model.coef_)[0]
b = model.intercept_

print(w_coeffs)

print(model.intercept_)


[ 0.99971105  0.86310389  0.80299318 -1.29983596 -0.88281622]
[-0.35449221]


In [892]:
line = list(x_test[10])

In [893]:
line, w_coeffs, b[0]

([-0.6270717965666156,
  -0.7498071314502132,
  -0.9063537657727768,
  -0.46173278681778607,
  -0.5172256655088346],
 array([ 0.99971105,  0.86310389,  0.80299318, -1.29983596, -0.88281622]),
 -0.35449220697468387)

In [899]:
distance_into_probability(line, w_coeffs, b[0]) * 100

21.424108639452115

In [898]:
des = model.decision_function([x_test[10]])
1 / (1 + np.exp(-des[0]))

0.21424108639452125